In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import mysql.connector

In [10]:
df_ccaa = pd.read_csv('datos/energia_año_ccaa.csv', index_col=0)
df_peninsula = pd.read_csv('datos/energia_año_peninsula.csv', index_col=0)

# Pair Programming ETL Carga I

1 .Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [2]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña, plugin):
        
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña
        self.plugin = plugin

    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                        user="root",
                                        password=f'{self.contraseña}',
                                        auth_plugin = f'{self.plugin}')
        mycursor = mydb.cursor()

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            print(mycursor)
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)



    def crear_insertar_tabla(self,query):
    
        cnx = mysql.connector.connect(user='root', password=f"{self.contraseña}",
                                        host='127.0.0.1', database=f"{self.nombre_bbdd}", 
                                        auth_plugin = f'{self.plugin}')

        mycursor = cnx.cursor()
        
        try: 
            mycursor.execute(query)
            cnx.commit() 

        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

    def check_comunidades(self):

        mydb = mysql.connector.connect(user='root',
                                    password=f"{self.contraseña}",
                                    host='127.0.0.1',
                                    database=f"{self.nombre_bbdd}",
                                    auth_plugin = f'{self.plugin}')
        mycursor = mydb.cursor()

        query_existe_comunidades = f"""
                SELECT DISTINCT comunidad FROM comunidades
                """
        mycursor.execute(query_existe_comunidades)
        comunidades = mycursor.fetchall()
        return comunidades
    
    def sacar_id_comunidad(self, comunidad):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{self.contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{self.nombre_bbdd}",
                                       auth_plugin = f'{self.plugin}')
        mycursor = mydb.cursor()
        
        try:
            query_sacar_id = f"SELECT idcomunidad FROM comunidades WHERE comunidad = '{comunidad}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa ciudad en la BBDD y por lo tanto no te podemos dar su id. "
        
    def sacar_id_fecha(self, fecha):
        mydb = mysql.connector.connect(user='root', password=f'{self.contraseña}',
                                          host='127.0.0.1', database=f"{self.nombre_bbdd}",
                                          auth_plugin = f'{self.plugin}')
        mycursor = mydb.cursor()

        try:
            query_sacar_id = f"SELECT idfechas FROM fechas WHERE fecha = '{fecha}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
             return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "

In [3]:
query_fecha = """CREATE TABLE IF NOT EXISTS `energia`.`fechas` (
  `idfechas` INT NOT NULL AUTO_INCREMENT,
  `fecha` DATE NOT NULL,
  PRIMARY KEY (`idfechas`))
ENGINE = InnoDB;"""

In [4]:
query_comunidades = """ CREATE TABLE IF NOT EXISTS `energia`.`comunidades` (
  `idcomunidades` INT NOT NULL AUTO_INCREMENT,
  `comunidad` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`idcomunidades`))
ENGINE = InnoDB;"""

In [5]:
query_nacional = """CREATE TABLE IF NOT EXISTS `energia`.`nacional_renovable_no_renovable` (
  `idnacional_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NOT NULL,
  `tipo_energia` VARCHAR(45) NOT NULL,
  `valor` DECIMAL NOT NULL,
  `fechas_idfechas` INT NOT NULL,
  PRIMARY KEY (`idnacional_renovable_no_renovable`),
  INDEX `fk_nacional_renovable_no_renovable_fechas1_idx` (`fechas_idfechas` ASC) VISIBLE,
  CONSTRAINT `fk_nacional_renovable_no_renovable_fechas1`
    FOREIGN KEY (`fechas_idfechas`)
    REFERENCES `energia`.`fechas` (`idfechas`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;"""

In [6]:
query_comunidades_renovables_norenovables = """ CREATE TABLE IF NOT EXISTS `energia`.`comunidades_renovable_no_renovable` (
  `idcomunidades_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NOT NULL,
  `tipo_energia` VARCHAR(45) NOT NULL,
  `valor` DECIMAL NOT NULL,
  `fechas_idfechas` INT NOT NULL,
  `comunidades_idcomunidades` INT NOT NULL,
  PRIMARY KEY (`idcomunidades_renovable_no_renovable`),
  INDEX `fk_comunidades_renovable_no_renovable_fechas1_idx` (`fechas_idfechas` ASC) VISIBLE,
  INDEX `fk_comunidades_renovable_no_renovable_comunidades1_idx` (`comunidades_idcomunidades` ASC) VISIBLE,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_fechas1`
    FOREIGN KEY (`fechas_idfechas`)
    REFERENCES `energia`.`fechas` (`idfechas`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_comunidades1`
    FOREIGN KEY (`comunidades_idcomunidades`)
    REFERENCES `energia`.`comunidades` (`idcomunidades`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
ENGINE = InnoDB;"""

In [7]:
bbdd_energia = Cargar('energia', 'AlumnaAdalab', 'mysql_native_password')

In [8]:
bbdd_energia.crear_bbdd()

MySQLCursor: CREATE DATABASE IF NOT EXISTS energia;


In [9]:
bbdd_energia.crear_insertar_tabla(query_fecha)
bbdd_energia.crear_insertar_tabla(query_nacional)
bbdd_energia.crear_insertar_tabla(query_comunidades)
bbdd_energia.crear_insertar_tabla(query_comunidades_renovables_norenovables)

In [12]:
df_ccaa.sample()

,value,percentage,datetime,tipo_energia,region,cod_comunidad
0,211290.04,0.98,2011-01-01,No renovable,Melilla,8745


In [13]:
for indice, fila in df_ccaa.iterrows():

    query_date_time = f"""INSERT INTO fechas (fecha)  
        VALUES ('{fila["datetime"]}');"""
    
    bbdd_energia.crear_insertar_tabla(query_date_time)

In [14]:
for indice, fila in df_ccaa.iterrows():

    query_ccaa = f"""INSERT INTO comunidades (comunidad)  
        VALUES ('{fila["region"]}');"""
    
    bbdd_energia.crear_insertar_tabla(query_ccaa)

In [15]:
bbdd_energia.check_comunidades()

[('Ceuta',),
 ('Melilla',),
 ('Andalucía',),
 ('Aragón',),
 ('Cantabria',),
 ('Castilla - La Mancha',),
 ('Castilla y León',),
 ('Cataluña',),
 ('País Vasco',),
 ('Principado de Asturias',),
 ('Comunidad de Madrid',),
 ('Comunidad Foral de Navarra',),
 ('Comunitat Valenciana',),
 ('Extremadura',),
 ('Galicia',),
 ('Illes Balears',),
 ('Canarias',),
 ('Región de Murcia',),
 ('La Rioja',)]

In [17]:
df_ccaa.sample()

,value,percentage,datetime,tipo_energia,region,cod_comunidad
0,9667000.41,0.73,2014-01-01,No renovable,Principado de Asturias,11


In [18]:
for indice, fila in df_ccaa.iterrows():

    query_energia_ccaa = f"""INSERT INTO comunidades (porcentaje, tipo_energia, valor)  
        VALUES ('{fila["percentage", "tipo_energia", "value"]}');"""
    
    bbdd_energia.crear_insertar_tabla(query_energia_ccaa)

KeyError: 'key of type tuple not found and not a MultiIndex'